In [1]:
from typing import *
import sys
import importlib
import inspect
import math
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pickle
import time

In [2]:
from pprint import pprint
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, IBMQ, execute
from qiskit.transpiler.passes import RemoveBarriers
from qiskit.circuit import Parameter
from qiskit.compiler import transpile
import qiskit.ignis.mitigation as mit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_14668/1908178760.py:7: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  import qiskit.ignis.mitigation as mit


In [3]:
shots = 1 << 13
max_iter = 10
min_size = 1
max_size = 30

In [4]:
raw_hist_list_list = []
meas_fitter_list_list = []
for num_qubits in range(min_size, max_size + 1):
    
    t1 = time.perf_counter()
    
    noise_model = NoiseModel()
    for qi in range(num_qubits):
        if qi >= 0:
            read_err = ReadoutError([[0.950, 0.050],[0.050,0.950]])
            noise_model.add_readout_error(read_err, [qi])
            
    raw_hist_list = []
    meas_fitter_list = []
    for i in range(max_iter):
        qc_y = QuantumCircuit(num_qubits)
        qc_y.measure_all()
        job = execute(qc_y, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model, seed_transpiler=i, seed_simulator=i)
        raw_hist_list.append(job.result().get_counts())
        
        qr = QuantumRegister(num_qubits)
        mit_pattern = [[i] for i in range(num_qubits)]
        meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
        job = execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model, seed_transpiler=100 * i, seed_simulator=100 * i)
        cal_results = job.result()
        meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
        meas_fitter_list.append(meas_fitter)
    
    raw_hist_list_list.append(raw_hist_list)
    meas_fitter_list_list.append(meas_fitter_list)
    
    t2 = time.perf_counter()
    
    print("size", num_qubits, "finished (t = ", t2 - t1, "s)")

size 1 finished (t =  0.80654033 s)
size 2 finished (t =  0.7537445119999999 s)
size 3 finished (t =  0.9774438499999998 s)
size 4 finished (t =  1.1721098950000002 s)
size 5 finished (t =  1.4013362010000003 s)
size 6 finished (t =  1.6234314240000005 s)
size 7 finished (t =  1.8523673079999998 s)
size 8 finished (t =  2.1205825469999997 s)
size 9 finished (t =  2.506936908 s)
size 10 finished (t =  2.734710570999999 s)
size 11 finished (t =  2.833866202000003 s)
size 12 finished (t =  3.156479138999998 s)
size 13 finished (t =  3.4264305519999994 s)
size 14 finished (t =  3.6987422320000007 s)
size 15 finished (t =  3.894418292000001 s)
size 16 finished (t =  4.177113133000006 s)
size 17 finished (t =  4.499878734999996 s)
size 18 finished (t =  4.871092001000001 s)
size 19 finished (t =  5.100744525000003 s)
size 20 finished (t =  5.410287621000002 s)
size 21 finished (t =  5.756722938000003 s)
size 22 finished (t =  6.112885437000003 s)
size 23 finished (t =  6.449070592999988 s)
s

In [6]:
with open("pkls/raw_hist_list_list.pkl", "wb") as f:
    pickle.dump(raw_hist_list_list, f)
with open("pkls/meas_fitter_list_list.pkl", "wb") as f:
    pickle.dump(meas_fitter_list_list, f)